# Análise comparativa

## 1. Preparação de dados

### 1.1 Configurações iniciais

Importação de bibliotecas

In [39]:
from pathlib import Path
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


Lendo o conjunto de dados

In [40]:
data_path = Path('../data/raw/hour.csv')
df = pd.read_csv(data_path)

Lendo dicionário de dados

In [41]:
dict_path = Path('../data/external/dicionario.csv')
df_dict = pd.read_csv(dict_path, sep=';')

### 1.2 Tratamento de dados

Para cada tipo de variável, será criado um pipeline para pré-processamento com os seguintes itens:
- Tratamento de dados discrepantes
- Tratamento de dados faltantes
- Codificação de variáveis
- Seleção de variáveis
- Normalização


Variável alvo

In [42]:
target_column = 'cnt'

Variáveis discretas

In [43]:
discrete_columns = (
    df_dict
    .query('Tipo == "Discreta" and Variável != @target_column')
    ['Variável']
    .to_list()
)

Pipeline

In [44]:
discrete_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ('missing', SimpleImputer(strategy='mean')), # Tratamento de dados faltantes
    # Seleção de variáveis
    ('normalization', MinMaxScaler()) # Normalização
])

Variáveis nominais

In [45]:
nominal_columns = (
    df_dict
    .query('Tipo == "Nominal"')
    ['Variável']
    .to_list()
)

Pipeline

In [46]:
nominal_preprocessor = Pipeline([
    # Tratamento de dados discrepantes
    ('missing', SimpleImputer(strategy='most_frequent')), # Tratamento de dados faltantes
    ('encoder', OneHotEncoder(sparse=False)), # Codificação de variáveis
    # Seleção de variáveis
    ('normalization', MinMaxScaler()), # Normalização    
])

Aplicando pré-processadores

In [47]:
preprocessing = ColumnTransformer([
    ("discrete", discrete_preprocessor, discrete_columns),
    ("nominal", nominal_preprocessor, nominal_columns)
])